# Obstacle Avoidance 
**Full Coding At [Obstacle_Avoidance.py](Obstacle_Avoidance.py)** 
<br>
This is the documentation for Obstacle Avoidance (With Camera) 

***Libraries Used In The Script***
- Ultrasonic_sens  
- PCA9685_MC 
- Motor_Encoder  
- time 
- threading 
- picamera2 
- OpenCV
- libcamera 
<br>



**All the libraries are custom made libraraies thus installation is no needed**
- *Ultrasonic_sens* 
    - Used to obtained all the value of the ultrasonic ranger at once
- *PCA9685_MC* 
    - Used to controll the Motor Direction more easily using Servo Motor HAT 
- *Motor_Encoder* 
     - Used to obtained the value of the RPM of the motor (With OLED Function)
- *time*
    - Used for delay 

## Let's Start Coding ! 
### 1. Import all the Libraries 
- Threading (For Multithreading proces)
- OpenCV (To display the captured image)
- Picamer2 (To capture the image using the camera)
- Ultrasonic_sens  (To get the distance from the obstacle using ultrasonic ranger)
- PCA9685_MC (Control the motor and servo motor)
- Motor_Encoder  (get the encoder value )
- time (To set delay)
- libcamera (To control the Auto Focus fopr the camera)

In [ ]:
import threading 
import cv2 
from picamera2 import Picamera2
from Ultrasonic_sens import Ultrasonic 
from PCA9685_MC import Motor_Controller
from Motor_Encoder import Encoder 
import time 
from libcamera import controls # Uncommand if not using Raspbery Pi Camera Module 3



### 2. Initialise the libraries in the *init* Function 
- Create global variable so that other function can also access to it. 
  ```python 
  global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist, picam2, frame_lock, shutdown_event, isInit
  ```
- Make sure the initaliation process is done 
  ``` python 
  isInit = False 
  ```
  
- If the initialisation is NOT **True** (Not be initialise) run the following:
  - Initialise the motor encoder library with the **ODISPLAY** enabled 
  - Initialise the ultrasonic sensors 
  - Initialise the motor library 
  - Define the servo chanel for the vertical and horizontal servo PanTilt HAT
  - Set the position of the camera 
  - Set the speed at 20 
  - set the rotation speed  at 15 
  - Set the threshold to 30 
  - Set the minimum threshold distance to 10 
  - Intialise the camera 
  - Configure the camera 
  - Start the camera 
  - Set the AutoFocus mode to continous 
  - Set Frame Lock for multi threading 
  - Define shutdown event for threading 
  - Set the initialation flag to **True**  

- If the initialation process is done, this function will be skip. 
  

In [ ]:
def init():
    global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist, picam2, frame_lock, shutdown_event, isInit
    isInit = False

    if not isInit: 
        
        enc = Encoder(ODISPLAY=True)
        ultrasonic = Ultrasonic()
        Motor = Motor_Controller()
        vertical = 0
        horizontal = 1
        Motor.servoPulse(horizontal, 1250)
        Motor.servoPulse(vertical, 1050)
        Speed = 20
        rotation_speed = 50
        threshold = 30 
        min_thresh_dist = 10 
        picam2 = Picamera2() 
        picam2.configure(picam2.create_preview_configuration(main={"format": 'XRGB8888', "size": (640, 480)}))
        picam2.start()
        picam2.set_controls({"AfMode": controls.AfModeEnum.Continuous})
        frame_lock = threading.Lock()
        shutdown_event = threading.Event()
        isInit = True 


### 3. The Capture image Function `def capture_frame()` 
- This function will capture the image and display it on a window using OpenCV and camera. 
- Access to the global variable 
```py
    global picam2, frame_lock, shutdown_event
``` 
- The function wil be execute if the `shutdown_event` is NOT set 
```py
    while not shutdown_event.is_set():
```
- With the multithreading frame lock, capture the image
```py 
    with frame_lock:
            frame = picam2.capture_array()
```
- Using OpenCV to show the captured image
```py
    cv2.imshow("Frame", frame)
```
- When the button `q` is pressed, quit the programe.
```py
    if cv2.waitKey(1) & 0xFF == ord('q'):
            shutdown_event.set()  # Signal to stop the program
            break 
    cv2.destroyAllWindows()  # Cleanup the OpenCV windows
```

In [ ]:
# Capture and Display Frame 
def capture_frame():
    global picam2, frame_lock, shutdown_event
    while not shutdown_event.is_set():
        with frame_lock:
            frame = picam2.capture_array()
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            shutdown_event.set()  # Signal to stop the program
            break 
    cv2.destroyAllWindows()  # Cleanup the OpenCV windows

### 4. The *Obstacle Avoid Function* `def obstacle_avoid(left. front. right)` 
- This is the logic part that wil process the data get from the sensors and determine the diurection of the obstacle. 
- In this function the function will collect the data from the left, front and right ultrasonic sensor to determine the distance of the obstacle from the sourounding. 
- Access to the global variables 
  ``` python
  global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist
  ```
- If the value of the fron sensor is less than the given threshold (30) Multiple condition will be triggred. 
  - If the front value is less than the minimum threshold value (20), the Car will go backward with the speed of 20 
  - If the left and right sensor are less than the minumum vlue, the Car will Reverse untill the values re more than minimum threshold value (20)
  - If obstacle at the left ensor is less than the threshold value, the car will turn right. 
  - If the obstacle at the right sensor is less than the threshold value, the car will turn left. 
  - Else, the Car will brake. 

- Else if the left sensor is less than the threshold value but the front sensor still have some sapce,  the car will turn Right
- Else if the  right sensor is less than the threshold value but the front sensor still have some space, the car will turn left 
- **The two else if logic above is trying to make sure the car is in the center if two obstacle*

- If the left and the right sensor are all less than threshold but the foront sensor is more than the threshold value, the care will proceed forward.  
  
- If all the condition above is not meet, the car will brake 

In [ ]:
def obstacle_Avoid(left, front, right):
    global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist
    
    if front < threshold:
        if front <= min_thresh_dist:
            Motor.Backward(Speed)
            time.sleep(0.1)
            Motor.Brake()
        elif left < min_thresh_dist and right < min_thresh_dist:
            Motor.Backward(Speed)
            time.sleep(0.1)
            Motor.Brake()
        elif left < threshold:
            Motor.Clock_Rotate(rotation_speed)
            time.sleep(0.5)
            Motor.Brake()
        elif right < threshold:
            Motor.AntiClock_Rotate(rotation_speed)
            time.sleep(0.5)
            Motor.Brake()
        else:
            Motor.Backward(Speed)
            time.sleep(0.1)
            Motor.Brake()
    
    elif left < threshold:
        Motor.Clock_Rotate(rotation_speed)
        time.sleep(1)
        Motor.Brake()
    
    elif right < threshold:
        Motor.AntiClock_Rotate(rotation_speed)  
        time.sleep(1)
        Motor.Brake()
    
    elif right < threshold and left < threshold and front > threshold :
            Motor.Forward(Speed)
            time.sleep(0.1)
            Motor.Brake()
    else:
        Motor.Brake()

### 5. The `main` function 
- Get the global variables by `global` 
    ``` python
    global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist, picam2, shutdown_event
    ```

- Coll for the `init()` function to make sure all theh libraries are initialised. 
- Start the camera capture thread using mutithreading 
    ```py
    capture_thread = threading.Thread(target=capture_frame)  # Start the capture frame thread
    capture_thread.start()
    ```
- `While True` is a technique to create a loop in python. 
    - Obtain the encoder value. if the **ODISPLAY** is true, the encoder value wil be display on the OLED. 
    - Display the value of the left, front and right ultrasonic sensor 
    - To make sure the variable that holds the sensor's value is valid, the if statem,ent will checkis the value is None or a valud value.
        ```python 
        if (left and front and right is not None) 
        ``` 
    - If the value from the sensor is not valid *(None)* 
        - The Car will brake and wait for the value of the sensor to become valid (not None). 
    - This step is to prevent the car moving without detecting the obstacle from the enviroment. 
    

In [ ]:
def main(): 
    global enc, ultrasonic, Motor, Speed, rotation_speed, threshold, min_thresh_dist, picam2, shutdown_event
    
    init()
    print("Program Started")
    
    capture_thread = threading.Thread(target=capture_frame)  # Start the capture frame thread
    capture_thread.start()

    while not shutdown_event.is_set():
        enc.encoder()
        left, front, right = ultrasonic.distances()
        time.sleep(0.1)
        if left is not None and front is not None and right is not None: 
            print("Left: {:.2f}".format(left)) 
            print("Front: {:.2f}".format(front))
            print("Right: {:.2f}".format(right)) 
            print(" ")
            obstacle_Avoid(left, front, right)
            time.sleep(0.1)
        else:
            print("No data received")
            Motor.Brake()
            time.sleep(1)

### 5. Wraping Up 
- If the program is started, run the `main()` function 
    ```py
    try:
        if __name__ == "__main__":
            main()
    ```
- Keyboard Inturrupt is detected (Ctrl + C) 
    ```py 
    except KeyboardInterrupt:
        shutdown_event.set()
        Motor.cleanup()
        enc.stop()
        print("Program Terminated")
    ```
- When the programe ends (Finally)
    ```py
    finally: 
        shutdown_event.set()
        enc.stop()
        capture_thread.join()
        cv2.destroyAllWindows()
        exit()
        print("Program Terminated")
    ```



In [ ]:
try:
    if __name__ == "__main__":
        main()
            

except KeyboardInterrupt:
    shutdown_event.set()
    Motor.cleanup()
    enc.stop()
    print("Program Terminated")

finally: 
    shutdown_event.set()
    enc.stop()
    capture_thread.join()
    cv2.destroyAllWindows()
    exit()
    print("Program Terminated")